# 05. 파이썬을 이용한 Word2Vec (Using Gensim)

* 싸이그래머 / 어바웃 파이썬
* 김무성

# 차례
* Word2Vec 
* Gensim을 이용한 Word2Vec 간단 예제
* Gensim을 이용한 한국어 Word2Vec 예제
* Gensim과 plotly를 이용한 한국어 Word2Vec 시각화(t-SNE)
* Gensim과 Tensorboard를 이용한 한국어 Word2Vec 시각화(t-SNE)
* Gensim을 이용한 Word2Vec Online Update (새로운 코러스 기존 모델에 반영하는 업데이트)

# Word2Vec
* [1] Brain's Pick : 단어 간 유사도 파악 방법 - https://brunch.co.kr/@kakao-it/189
* [5] word2vec 관련 이론 정리 - https://shuuki4.wordpress.com/2016/01/27/word2vec-%EA%B4%80%EB%A0%A8-%EC%9D%B4%EB%A1%A0-%EC%A0%95%EB%A6%AC/
* [6] Vector Representations of Words - https://www.tensorflow.org/tutorials/word2vec

<img src="05_figures/cat1.png" width=600 />
<img src="05_figures/cat2.png" width=600 />
<img src="05_figures/w2v.png" width=600 />
<img src="05_figures/we.png" width=600 />
<img src="05_figures/tsne.png" width=600 />

--------------------------

# Gensim을 이용한 Word2Vec 간단 예제

* [2] models.word2vec – Deep learning with word2vec - https://radimrehurek.com/gensim/models/word2vec.html

### 영어 예시

In [ ]:
import gensim

In [ ]:
# Data
documents = [["my", "name", "is", "jamie"], ["jamie", "is", "cute"]]

In [ ]:
# 모델 초기화
model = gensim.models.Word2Vec(min_count=1) # 예) Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

In [ ]:
# 모델 사전 만들기
model.build_vocab(documents)

In [ ]:
# 학습
model.train(documents, total_examples=len(documents), epochs=10)

In [ ]:
# 생성된 모델 저장 및 불러오기 - 이것은 나중에 이 모델을 다시 활용하려할 때 써보기. 
fname = 'mymodel.wv'
model.save(fname)
my_model = gensim.models.Word2Vec.load(fname)  

In [ ]:
%ls mymodel.wv

In [ ]:
# 단어의 벡터값 얻기 
model.wv['name']  # numpy vector of a word

### 한글 예시

##### 다음 문서들을 이용해서 데이터를 만들자. 
* https://gasazip.com/view.html?no=614736
* https://gasazip.com/1224697
* https://gasazip.com/view.html?no=599082
* https://gasazip.com/view.html?no=645465
* http://gasazip.com/view.html?no=643505
* https://gasazip.com/view.html?no=615362

In [ ]:
# -- 코딩 / 아래는 예시 - 우선은 전처리 하지 말고 바로 해보자.

# Data
# ---- 여기서부터 -------------------------------
d1 = ["매일", "울리는", "벨벨벨"]
d2 = ["Sign을", "보내", "signal 보내"]

documents = []
documents.append(d1)
documents.append(d2)
# 여기까지를, , 실습을 위해 제대로 바꾸면 된다 ---------

In [ ]:

# 모델 초기화
model = gensim.models.Word2Vec(min_count=1) # 예) Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

# 모델 사전 만들기
model.build_vocab(documents)

# 학습
model.train(documents, total_examples=len(documents), epochs=10)

# 단어의 벡터값 얻기 
model.wv['벨벨벨']  # numpy vector of a word

# Gensim을 이용한 한국어 Word2Vec 예제

* [3] 한국어 Word2Vec - http://blog.theeluwin.kr/post/146591096133/%ED%95%9C%EA%B5%AD%EC%96%B4-word2vec


##### 다음 문서들을 이용해서 데이터를 만들자. 
* https://gasazip.com/view.html?no=614736
* https://gasazip.com/1224697
* https://gasazip.com/view.html?no=599082
* https://gasazip.com/view.html?no=645465
* http://gasazip.com/view.html?no=643505
* https://gasazip.com/view.html?no=615362    

### 실습
참고자료 [3]을 보고 직접 해보자. 형태소 분석도 해서.

In [ ]:
# -- coding

# Gensim과 plotly를 이용한 한국어 Word2Vec 시각화(t-SNE)
* plotly로 시각화 함수 만들기
* 간단한 모델 결과 시각화
* konlpy 내장된 국회 관련 데이터를 이용한 모델 시각화 

#### 참고
* [7] Word2Vec Tutorial - https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/word2vec.ipynb
* [8] 한글 word2vec을 이용한 유사도 분석 - http://www.neuromancer.kr/t/word2vec/487

## plotly로 시각화 함수 만들기

In [ ]:
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#logging.getLogger().setLevel(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction
import numpy as np                                  # array handling

from plotly.offline import init_notebook_mode, iplot, plot
import plotly.graph_objs as go

def reduce_dimensions(model, plot_in_notebook = True):

    num_dimensions = 2  # final num dimensions (2D, 3D, etc)

    vectors = []        # positions in vector space
    labels = []         # keep track of words to label our data again later
    for word in model.wv.vocab:
        vectors.append(model[word])
        labels.append(word)


    # convert both lists into numpy vectors for reduction
    vectors = np.asarray(vectors)
    labels = np.asarray(labels)
    
    # reduce using t-SNE
    vectors = np.asarray(vectors)
    logging.info('starting tSNE dimensionality reduction. This may take some time.')
    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
        
    # Create a trace
    trace = go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='text',
        text=labels
        )
    
    data = [trace]
    
    logging.info('All done. Plotting.')
    
    if plot_in_notebook:
        init_notebook_mode(connected=True)
        iplot(data, filename='word-embedding-plot')
    else:
        plot(data, filename='word-embedding-plot.html')

## 간단한 모델 결과 시각화

In [ ]:
reduce_dimensions(model)

## konlpy 내장된 국회 관련 데이터를 이용한 모델 시각화 

In [ ]:
#load from kobill 
from konlpy.corpus import kobill
docs_ko = [kobill.open(i).read() for i in kobill.fileids()]

In [ ]:
#Tokenize
from konlpy.tag import Twitter; t = Twitter()
#pos = lambda d: ['/'.join(p) for p in t.pos(d)]
pos = lambda d: [p[0] for p in t.pos(d) if 'Noun' in p] # 명사만
texts_ko = [pos(doc) for doc in docs_ko]

In [ ]:
# 모델 초기화
model = gensim.models.Word2Vec(min_count=1) # 예) Word2Vec(sentences, size=100, window=5, min_count=5, workers=4)

# 모델 사전 만들기
model.build_vocab(texts_ko)

# 학습
model.train(texts_ko, total_examples=len(texts_ko), epochs=10)

In [ ]:
reduce_dimensions(model)

# Gensim과 Tensorboard를 이용한 한국어 Word2Vec 시각화(t-SNE)

#### 참고
* [9] TensorBoard Visualizations - https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Tensorboard_visualizations.ipynb

In [ ]:
# -- 업데이트 예정

# Gensim을 이용한 Word2Vec Online Update (새로운 코러스 기존 모델에 반영하는 업데이트)

#### 참고
* [10] Online word2vec tutorial - https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/online_w2v_tutorial.ipynb

In [ ]:
# -- 업데이트 예정 

# 참고자료
* [1] Brain's Pick : 단어 간 유사도 파악 방법 - https://brunch.co.kr/@kakao-it/189
* [2] models.word2vec – Deep learning with word2vec - https://radimrehurek.com/gensim/models/word2vec.html
* [3] 한국어 Word2Vec - http://blog.theeluwin.kr/post/146591096133/%ED%95%9C%EA%B5%AD%EC%96%B4-word2vec
* [4] Word2Vec으로 문장 분류하기 - https://ratsgo.github.io/natural%20language%20processing/2017/03/08/word2vec/
* [5] word2vec 관련 이론 정리 - https://shuuki4.wordpress.com/2016/01/27/word2vec-%EA%B4%80%EB%A0%A8-%EC%9D%B4%EB%A1%A0-%EC%A0%95%EB%A6%AC/
* [6] Vector Representations of Words - https://www.tensorflow.org/tutorials/word2vec
* [7] Word2Vec Tutorial - https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/word2vec.ipynb
* [8] 한글 word2vec을 이용한 유사도 분석 - http://www.neuromancer.kr/t/word2vec/487
* [9] TensorBoard Visualizations - https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Tensorboard_visualizations.ipynb
* [10] Online word2vec tutorial - https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/online_w2v_tutorial.ipynb